---
title: "Visualization (Data Transformation)"
author: "Peter Ganong and Maggie Shi"
date: today
date-format: long
format: 
    html:
        echo: true
        toc: false
---

<!--
    beamer:
        echo: true
        aspectratio: 169
        theme: default
        toc: true
        header-includes: \renewcommand{\tightlist}{\setlength{\itemsep}{5ex}\setlength{\parskip}{0pt}}
            \setbeamertemplate{footline}[frame number] 
            -->



# introduction

## roadmap

* putting this lecture in context
* `movies` dataset
    * load data
    * `shape`
    * `head()`

(no summary at end of this section)

## putting this lecture in context

* Fundamental problem in data visualization: in most cases, you do not want to show every single data point in your dataset. 
* Instead, you want to extract patterns which you (the analyst) think are interesting. 
* This lecture explores methods for *transforming* data, focusing on aggregation.
* One nice thing about Altair is that it nudges you to aggregate. 
    * One example: if you try to make a plot with 10,000 dots, it will give you an error `MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).`
    * Help file: "This is not because Altair cannot handle larger datasets, but it is because it is important for the user to think carefully about how large datasets are handled. "
    * More details [here](https://altair-viz.github.io/user_guide/large_datasets.html)
* This lecture mostly follows Chapter 3 in the data visualization book (skip parts of section 3.2, which we will then come back to in lectures 4 and 5)

## load packages

In [ ]:
import pandas as pd
import altair as alt

## movies dataset

In [ ]:
movies_url = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'

In [ ]:
#| eval: false
movies = pd.read_json(movies_url)

In [ ]:
#| echo: false
movies = pd.read_json('data/movies.json')

In [ ]:
movies.shape

With 3201 movies, we are going to need to do some transformation if we want to uncover any patterns in the data!

## `head()`

In [ ]:
movies.head(5)

# Scatter plots and binning

## Scatter plots and binning: roadmap
* scatter plots
* binning

## scatter plot
* **Rotten Tomatoes** ratings are determined by taking "thumbs up" and "thumbs down" judgments from film critics and calculating the percentage of positive reviews.
* **IMDB ratings** are formed by averaging scores (ranging from 1 to 10) provided by the site's users.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q'),
    alt.Y('IMDB_Rating:Q')
)

## scatter plot -- add `bin=True`

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Y('IMDB_Rating:Q')
)

## scatter plot -- 20 bins

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('IMDB_Rating:Q')
)

<!-- PG to SSS this section is really underwhelming. it's not incorrect, it just sucks. it is completely unclear what the point is. We might consider deleting it entirely. Or binning on both the x and the y-variable so at least there's a bit more of a payoff-->

# Aggregation

## Aggregation: roadmap

In previous lectures, we actually already saw aggregation via `average()` and `min()`. We just didn't talk explicitly about that step. Now, we examine it more carefully.

* `average()`
* interquartile range
* do-pair-share

The Altair documentation includes the [full set of available aggregation functions](https://altair-viz.github.io/user_guide/encodings/index.html#aggregation-functions).


## `average()`


In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N')
)

This plot is fine, but hard to interpret takeaways quickly.

Discussion Question: what does the y-axis seem to be sorted on? Why?

What should we do? Sort the bars vertically, but by what's on the x-axis. 

But we haven't mixed X and Y encodings together, yet. What's the best way to figure out how? Ask [ChatGPT](https://chatgpt.com/share/67027f83-5c0c-800a-8253-3c4b4f074dce).

## `average()` with `sort(...)`

Now it's clear which movie types are most and least popular


In [ ]:
#Attribution: ChatGPT
#Query: I have the following bar chart code in Altair [...] 
# I want to sort the bars by the X encoding (average rotten tomatoes rating). How can I do that?
chart1  = alt.Chart(movies_url).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N', 
        sort=alt.EncodingSortField(
            op='average', 
            field='Rotten_Tomatoes_Rating', 
            order='descending'
        )
    )
)
chart1

Discussion question -- Why is "how to sorting the order of bars" such a great problem to submit to ChatGPT?


## Interquartile range


In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('q1(Rotten_Tomatoes_Rating):Q'),
    alt.X2('q3(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N', sort=alt.EncodingSortField(
        op='median', field='Rotten_Tomatoes_Rating', order='descending')
    )
)

Discussion question -- What can you learn from the interquartile range plot that you could not learn from the plot with just `average()`?


## Case study: when are the highest grossing films?


In [ ]:
movies_gross = movies[['US Gross', 'Release Date']]
movies_gross.head()

## a first pass

obviously we need to aggregate. 

also: what bug in the data does this plot reveal?


In [ ]:
alt.Chart(movies_url).mark_line().encode(
    alt.X('Release_Date:T'),
    alt.Y('US_Gross:Q')
)

## do-pair-share 

What time of year are the highest grossing films released? Aggregate both the x- and the y-variables.

Hint: if you aren't sure how to aggregate the x-variable, go back to the first visualization lecture as an example.

1. *Do* -- make a plot on your own

2. *Pair* -- compare your results with person next to you

3. *Share* -- discuss results as a class

In [ ]:
chart2 = alt.Chart(movies_url).().encode(
    alt.X('month(Release_Date):T'),
    alt.Y('max(US_Gross):Q')
)
chart2

## Aggregation: summary

* Quantitative beyond `count()` and `average()`
  * Distribution: `min()`, `q1()`, `median()`, `q3()`, `max()`
  * Dispersion: `variance()`, `stdev()`, `distinct()`
  * Bootstrap confidence intervals: `ci0()`, `ci1()` 
* Dates: see prior slide

# Advanced data transformation

## Advanced data transformation: roadmap

* Two ways to aggregate data in Altair: within the encoding itself, or using a top level aggregate transform. 
* Doing it in the encoding is fine for simple transformations, but for advanced transformations, we'll have to define it separately

* `transform_calculate()`
* `transform_filter()`
* do-pair-share
* `transform_aggregate()`
* `transform_window()`

These are all written in the [Vega expression language](https://vega.github.io/vega/docs/expressions/).


## Advanced data transformation: connection to packages you might already know

Purpose | Vega | `pandas` equivalent | R `dplyr` equivalent (`df %>% ...`) | 
| --- | --- | --- | 
Define a new variable | `transform_calculate()` | `df['new_col']` | `mutate()` |
Filter to subset of rows | `transform_filter(cond)` | `df.loc[cond]` | `filter(cond)` |
Aggregate function - reduces number of rows down to one per group | `transform_aggregate(groupby(...))` | `df.groupby('A').agg('mean')` | `group_by(A) |> summarise(mean(...))` |
Window function - transform across multiple rows, keeps same num. of rows) | `transform_window(sum())` | `df['values'].cumsum()`  | `mutate(cumsum())` |

One way to think of these verbs is that they are fundamental to any data analysis project and so in any/every language you learn, you need to know how to do these.

## connection to prior material

You already know how to do these all in `pandas` and in `dplyr` so it is not conceptually new. 

Why bother doing it in Altair/Vega?

**Exploratory data analysis** can be done faster in Altair: manipulate data and plot simulataneously

## `transform_calculate` case study redux: what time of year do US movies make money abroad? 

In [ ]:
alt.Chart(movies_url).mark_area().transform_calculate(
    NonUS_Gross='datum.Worldwide_Gross - datum.US_Gross'
).encode(
    alt.X('month(Release_Date):T'),
    alt.Y('median(NonUS_Gross):Q')
)

* `datum` is how you reference the underlying dataset within a transformation expression
* `transform_calculate()` uses expressions for writing basic formulas
    * Math functions: `min()`, `random()`, `round()`
    * Statistical functions: `sampleNormal()`, `sampleUniform()`
    * Date-time functions: `date()`, `year()`, `month()`
    * String functions: `length()`, `lower()`,`substring()`
    * Full list [here](https://vega.github.io/vega/docs/expressions/)


## `transform_filter` show just movies before 1970


In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q'),
    alt.Y('IMDB_Rating:Q')
).transform_filter('year(datum.Release_Date) < 1970')

## Do-pair-share
- Make two plots that compare ratings before and after 1970
    - Plot before and after 1970 on one plot
        - Create a categorical variable to indicate whether an observation is from before or after 1970. 
        - Encode the color of the mark depending on the value of that categorical variable
    - Append together two plots side by side: a scatter plot of ratings before 1970 and after 1970
- Which do you prefer and why?


## `transform_aggregate` recap from earlier in lecture


In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N')
)

## `transform_aggregate` what's happening under the hood
The previous example did the aggregation directly in the encoding. We can also express this in a more verbose way, with `transform_aggregate()`


In [ ]:
alt.Chart(movies_url).mark_bar().transform_aggregate(
    groupby=['Major_Genre'],
    Average_Rating='average(Rotten_Tomatoes_Rating)'
).encode(
    alt.X('Average_Rating:Q'),
    alt.Y('Major_Genre:N')
)

Discussion question -- If prior two code blocks have identical output, which version is better (and why)?




## `transform_window`: case study: who are the top 20 grossing directors of all time?

Start by summing `Worldwide_Gross` for each director, and then plotting in descending order. 


In [ ]:
alt.Chart(movies_url).mark_bar().transform_aggregate(
    Gross='sum(Worldwide_Gross)',
    groupby=['Director']
).encode(
    alt.X('Gross:Q'),
    alt.Y('Director:N', sort=alt.EncodingSortField(
        op='max', field='Gross', order='descending'
    ))
)

That's a lot of directors! Let's restrict to the top 20 -> `transform_window()` + `transform_filter`

*Side note: when do we know when to use `transform_window()` vs. `transform_aggregate()`?
* `transform_aggregate()`: we want to reduce the number of rows (e.g., one row per group)
* `transform_window()`: we want to keep the same number of rows

In this case, we want to know the *ranking* of each row, but not reduce the total number of rows. So, use `transform_window()`.


In [ ]:
alt.Chart(movies_url).mark_bar().transform_aggregate(
    Gross='sum(Worldwide_Gross)',
    groupby=['Director']
).transform_window(
    Rank='rank()',
    sort=[alt.SortField('Gross', order='descending')]
).transform_filter(
    'datum.Rank <= 20'
).encode(
    alt.X('Gross:Q'),
    alt.Y('Director:N', sort=alt.EncodingSortField(
        op='max', field='Gross', order='descending'
    ))
)

`null` is not a director, and we certainly don't want to say they're the highest-grossing director. So let's remove that -> `transform_filter()` again


In [ ]:
alt.Chart(movies_url).mark_bar().transform_aggregate(
    Gross='sum(Worldwide_Gross)',
    groupby=['Director']
).transform_window(
    Rank='rank()',
    sort=[alt.SortField('Gross', order='descending')]
).transform_filter(
    'datum.Rank <= 20'
).transform_filter(
    'datum.Director != null'
).encode(
    alt.X('Gross:Q'),
    alt.Y('Director:N', sort=alt.EncodingSortField(
        op='max', field='Gross', order='descending'
    ))
)

**Question**: How many directors are displayed now? Why?


We need to remove `null` *before* ranking and filtering. Our final graph:

In [ ]:
alt.Chart(movies_url).mark_bar().transform_filter(
    'datum.Director != null'
).transform_aggregate(
    Gross='sum(Worldwide_Gross)',
    groupby=['Director']
).transform_window(
    Rank='rank()',
    sort=[alt.SortField('Gross', order='descending')]
).transform_filter(
    'datum.Rank < 20'
).encode(
    alt.X('Gross:Q'),
    alt.Y('Director:N', sort=alt.EncodingSortField(
        op='max', field='Gross', order='descending'
    ))
)

## `transform_window`: in-class exercise

Steven Spielberg has been quite successful in his career! However, showing sums might favor directors who have had longer careers, and so have made more movies and thus more money. 

What happens if we change the choice of aggregate operation? Who is the most successful director in terms of  `average` gross per film? Modify the aggregate transform above!




## Advanced data transformation: summary
Purpose | Vega | `pandas` equivalent | R `dplyr` equivalent (`df %>% ...`) | 
| --- | --- | --- | 
Define a new variable | `transform_calculate()` | `df['new_col']` | `mutate()` |
Filter to subset of rows | `transform_filter(cond)` | `df.loc[cond]` | `filter(cond)` |
Aggregate function - reduces number of rows down to one per group | `transform_aggregate(groupby(...))` | `df.groupby('A').agg('mean')` | `group_by(A) |> summarise(mean(...))` |
Window function - transform across multiple rows, keeps same num. of rows) | `transform_window(sum())` | `df['values'].cumsum()`  | `mutate(cumsum())` |


Finally, Altair actually has 19 transformation methods (and counting...) and we have only covered four of them. Read about the rest of them [here](https://altair-viz.github.io/user_guide/transform/index.html).